<a href="https://colab.research.google.com/github/gksals8378/Space_Radio_Disaster_Forecast/blob/main/keras_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/data/0_Preporcessed/filled_data_sarimax.csv')

In [ ]:
df

,date,flux
0,2008-01-01,94.800000
1,2008-01-02,96.930000
2,2008-01-03,97.710000
3,2008-01-04,90.090000
4,2008-01-05,90.740000
...,...,...
5474,2022-12-27,120.425000
5475,2022-12-28,125.205000
5476,2022-12-29,104.286257
5477,2022-12-30,128.634000


In [ ]:
split = round(5479 * 0.8)
split

4383

In [ ]:
df['date'] = pd.to_datetime(df['date'])

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

df['flux'] = scaler.fit_transform(df['flux'].values.reshape(-1, 1))

In [ ]:
# 시계열 데이터를 sequence로 변환
sequence_length = 10
sequences_X, sequences_y = [], []

for i in range(len(df) - sequence_length):
    sequences_X.append(df['flux'].iloc[i:i+sequence_length].values)
    sequences_y.append(df['flux'].iloc[i+sequence_length])


In [ ]:
X = np.array(sequences_X)
y = np.array(sequences_y)

In [ ]:
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

print(len(X_train))
print(len(X_test))
print(len(y_train))
print(len(y_test))

4383
1086
4383
1086


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense
from tensorflow.keras.losses import MeanSquaredError

model = Sequential()
model.add(SimpleRNN(units=32, input_shape=(sequence_length, 1)))
model.add(Dense(1, activation='linear'))

In [ ]:
# 'adam' LR defalut 0.001
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
X_train = X_train.reshape(-1, sequence_length, 1)
X_test = X_test.reshape(-1, sequence_length, 1)

In [ ]:
model.fit(X_train, y_train, epochs=100, batch_size=16)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("Root Mean Squared Error :", rmse)

Root Mean Squared Error : 0.07781596962539933


# df_test


In [ ]:
df_test = pd.read_csv('/content/drive/MyDrive/data/AI_우주관측_경진대회/ai_competition_test.csv')

In [ ]:
df_test['date'] = pd.to_datetime(df_test['date'])

df_test['flux'] = scaler.fit_transform(df_test['flux'].values.reshape(-1, 1))

In [ ]:
import numpy as np

# sequences_X 초기화
sequence_length = 10
sequences_X = []

# 시계열 데이터를 sequence로 변환
for i in range(len(df_test) - sequence_length):
    sequences_X.append(df_test['flux'].iloc[i:i+sequence_length].values)

# 시퀀스 데이터의 길이를 10으로 고정
max_sequence_length = 10

# 패딩을 추가하거나 잘라내기
X_padded = []
for sequence in sequences_X:
    if len(sequence) < max_sequence_length:
        # 패딩 추가
        padded_sequence = np.pad(sequence, (0, max_sequence_length - len(sequence)))
        X_padded.append(padded_sequence)
    else:
        # 최대 길이로 잘라내기
        X_padded.append(sequence[:max_sequence_length])

# NumPy 배열로 변환 후 3D 형태로 재구성
X_ = np.array(X_padded)
X_ = X_.reshape(-1, max_sequence_length, 1)

In [ ]:
len(X_)

5469

In [ ]:
y_pred = model.predict(X_)

171/171 [==============================] - 0s 2ms/step


In [ ]:
# 실제값
y_true = df_test['flux'].iloc[sequence_length:]
y_true = y_true[:len(y_pred)]

y_pred = y_pred[:len(y_true)]

# RMSE 계산
rmse = np.sqrt(mean_squared_error(y_true, y_pred))
print("Root Mean Squared Error:", rmse)

Root Mean Squared Error: 0.10781088153767794


In [ ]:
model.save("RNN_model.h5")